# Bayesian Batch Adaptation Using "Cutting the Feedback"

#### Author:  Austin Talbot



In this notebook we demonstrate usage of our new method for batch adaptation using Bayesian statistics combined with a technique called cutting the feedback to make the method robust to non-balanced data (our method won't fail if somebody unfortunately puts all controls in one batch and all cases in another batch). But before we explain how our method works, we first describe the setup/assumptions



## Setup and assumptions

### Background
We assume that our data come from $J$ batches and for each batch we have $I_j$ samples which we denote $\{x_{ij}\}_{i=1:I_j}$. Since this is designed around proteomics data $J$ will be relatively large (at least 20ish) while $I_j$ will be relatively small (around 15). These measurements have $p$ covariates and are normally distributed. The problem we face is that each batch has additive noise which we denote $\theta_i\sim N(0,\Sigma_\theta)$, which corrupt our observations. Let our signal in reality be $\delta_{ij}$, which we estimate. Then our observational model is 
\begin{equation}
\begin{split}
x_{ij}=\delta_{ij} + \theta_i + \epsilon_{ij}, \epsilon_{ij}\sim N(0,\Sigma_\epsilon).
\end{split}
\end{equation}

Methods such as COMBAT simply subtract off the mean of the batch as the way of adjusting for batch effects. However, there are two problems with this. (1) With small sample sizes this is throwing out a substantial portion of the information (in an extreme case with $I_j=1$ you lose the entire signal. And worse, (2) if the data are not balanced (i.e. different numbers of cases and controls) the sample mean is going to be correlated with the main signal we want to estimate (how does illness impact the covariates). To translate to statistics, $\delta_{ij}$ are not $iid$.

Normally, we would be up a creek without a paddle with this kind of question. However, we do have one saving grace, in each batch we also record the output of a control sample $y_i$, which is measured in all batches. This is equivalent to saying that $\delta_{i}=c$, which means that
\begin{equation}
y_i = c + \theta_i + \epsilon_i,\epsilon_{i}\sim N(0,\Sigma_\epsilon)
\end{equation}
We can pretend $c$ is 0 by subtracting off the control mean. Roughly true if enough samples are measured.

The innate tempation is to simply subtract off the value of this control to account for batch effects. This is nice because the resulting value is an unbiased estimator of $\delta{ij}$. Proof:

\begin{equation}
E[x_{ij}-y_i] = E[\delta_{ij} + \theta_i + \epsilon_{ij} - \theta_i - \epsilon_i]=\delta_{ij} +E[\epsilon_{ij}-\epsilon_i] = \delta_{ij}
\end{equation}

However, this estimator of $\delta_{ij}$ has a higher variance than $x_{ij}$, in fact it doubles the variance. In the extreme case that there are no batch effects, you've essentially doubled the measurement error.

### Our method
Instead, the optimal estimator for $\delta_{ij}$ is 
\begin{equation}
\tilde{x}_{ij} = x_{ij} - w y_i,
\end{equation}
where $w\in[0,1]$. I'm not going to derive the formula but in the univeriate case $w=\sigma^2_\theta/(\sigma^2_\theta+\sigma_\epsilon^2)$. If we knew $\Sigma_\epsilon,\Sigma_\theta$ this would be straightforward and solved. However, we must estimate them given the data. Once we put priors on our parameters we can be traditional Bayesians for this problem. Our model is

\begin{equation}
\begin{split}
p(\Sigma_\epsilon)&=Wishart(\nu+p,I)\\
p(\Sigma_\delta)&=Wishart(\nu+p,I)\\
p(\Sigma_\theta)&=InverseWishart(\nu+p,I)\\
p(\theta_i|\Sigma_\theta)&=N(0,\Sigma_\theta)\\
p(y_i|\theta_i,\Sigma_\epsilon)&=N(0,\Sigma_\theta)\\
p(x_{ij}|\theta_i,\Sigma_\epsilon,\delta_{ij})&=N(\theta_i+\delta_{ij},\Sigma_\epsilon)\\
\end{split}
\end{equation}


### Cutting the feedback

There is one major problem from the above formulation, we do not want $x_{ij}$ to assume iid. To solve this issue we will use a technique called cutting the feedback. Note that our posterior $p(\theta_i|-)$ will naturally depend on $x_{ij}$, as the samples are informative of the batch effects. However, in our sampling scheme we will cut this dependence and only condition $\theta_i$ on $y_i$. This is known as cutting the feedback. This substantially reduces our certainty on $\theta_i$, particularly if we were to have large batch sizes. And it makes samplers a real pain, which is why $\Sigma_\theta$ is the odd ball out. However, we will benefit enormously in an application where all the controls are put in a single batch.



## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bystro.domain_adaptation.batch_bayesian import BatchAdaptationBayesian

/home/austin/anaconda3/envs/bystro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Generative Mechanism for Data Simulation

1. **Batch Effects**:  
   A total of `n_batches` are generated from a multivariate normal distribution with mean zero and covariance matrix `sig_theta * I_p`, where `I_p` is the identity matrix of size `p`:
  $
   \text{Batch Effects} \sim \mathcal{N}(0, \sigma_{\theta}^2 I_p)
   $

2. **Controls**:  
   The controls are generated by adding Gaussian noise with variance `sig_eps^2` to the batch effects for each batch:
   $
   \text{Controls}_i = \text{Batch Effects}_i + \sigma_{\epsilon} \mathcal{N}(0, 1)
   $
   for each batch $(i = 1, \dots, n_{\text{batches}})$.

3. **True Values**:  
   For each batch, the true values are drawn from a normal distribution with variance `sig_delta^2`:
   $
   \text{True Values}_i \sim \sigma_{\delta} \mathcal{N}(0, 1)
   $
   for each sample within a batch \(i = 1, \dots, n_{\text{batches}}\).

4. **Observed Data**:  
   The observed data for each batch is generated by adding the batch effect and Gaussian noise with variance `sig_eps^2` to the true values:
   $
   X_i = \text{True Values}_i + \text{Batch Effects}_i + \sigma_{\epsilon} \mathcal{N}(0, 1)
   $
   for each batch $(i = 1, \dots, n_{\text{batches}})$.


In [2]:
def simulate_data(n_batches=30, n_samples=15, p=3, seed=1993,sig_theta=1.0,sig_eps=1.0,sig_delta=1.0):
    rng = np.random.default_rng(seed)
    batch_effects = rng.multivariate_normal(
        mean=np.zeros(p), cov=sig_theta * np.eye(p), size=n_batches
    )   

    controls = batch_effects + sig_eps*rng.normal(size=(n_batches, p)) 
    true_vals = [sig_delta*rng.normal(size=(n_samples, p)) for i in range(n_batches)]
    X_list = [ 
        true_vals[i] + batch_effects[i] + sig_eps*rng.normal(size=(n_samples, p)) 
        for i in range(n_batches)
    ]   
    return X_list, controls, true_vals, batch_effects


## Steps for our Bayesian Batch Adaptation Model

### 1. Creating a BatchAdaptationBayesian Object
In Bystro (or similarly in scikit-learn), the first step is to choose the BatchAdaptationBayesian method for imputing missing data. This model corresponds to PCA from scikit learn. Any training options go in the creation method.

### 2. Fit Model Using the `fit_transform()` Method
Once you have your PPCADropout object, the next step is to fit the model given data. You do this by calling the `fit_transform()`. This will estimate $\Sigma_\epsilon,\Sigma_\theta,w$ and return the adapted measurments. `fit_transform` takes 2 inputs, a list of matrices corresponding to the measurments from your experiment, and a matrix of control samples. 

### 3. Transforming new data
After the `fit_transform()` method has been applied, you can account for batch effects in new data using the `transform()` method. This takes a new list and matrix of controls. Unlike `fit_transform`, it uses the previously-computed 

### Putting It All Together
The process with SoftImpute is straightforward:

- **Create**: You start by creating a BatchAdaptationBayesian object that will learn your model
- **Fit_transform**: You then adjust for batch effects
- **transform**: Remove batch effects in new data



## Demonstrating usage

We now demonstrate usage on synthetic data and compare our performance to ignoring batch effects and subtracting off the control samples.

In [3]:
def run_experiment(seed,sig_theta=1.0,sig_eps=1.0):
    X_list, controls, true_vals, batch_effects = simulate_data(seed=seed,sig_theta=sig_theta,sig_eps=sig_eps)

    model = BatchAdaptationBayesian(n_burn=500,n_samples=100)
    data_altered = model.fit_transform(X_list, controls)

    data_altered_stack = np.vstack(data_altered)
    data_original_stack = np.vstack(X_list)
    true_stack = np.vstack(true_vals)

    data_subtracted = [X_list[i] - controls[i] for i in range(len(X_list))]
    data_subtracted_stack = np.vstack(data_subtracted)

    error_adapted = np.mean((data_altered_stack - true_stack) ** 2)
    error_original = np.mean((data_original_stack - true_stack) ** 2)
    error_subtracted = np.mean((data_subtracted_stack - true_stack) ** 2)
    print('Ours',error_adapted)
    print('Ignored',error_original)
    print('Naive',error_subtracted)
    print(model.w)

    return model,X_list, controls, true_vals, batch_effects,data_subtracted,data_altered


## Comparable measurement noise and batch effects

In our first example, we show what happens when the measurement noise is comparable to the batch effects (both $\Sigma_\epsilon,\Sigma_\theta=I$). In this case, $w=0.5$. 

In [4]:
model,X_list, controls, true_vals, batch_effects,data_subtracted,data_altered = run_experiment(2021)

  0%|                                                   | 0/500 [00:00<?, ?it/s]/home/austin/anaconda3/envs/bystro/lib/python3.10/site-packages/pyro/distributions/lkj.py:16: FutureWarning: class LKJCorrCholesky(d, eta, validate_args=None) is deprecated in favor of LKJCholesky(dim, concentration, validate_args=None).
  warnings.warn(
100%|█████████████████████████████████████████| 100/100 [01:19<00:00,  1.25it/s]

Ours 1.692883930041417
Ignored 1.8987285660395787
Naive 2.1695650063784666
[0.22250731 0.26031832 0.44168912]


So the MSE of our estimates of $\delta$ was $1.69$ as opposed to the case where we ignore batch effects ($1.89$) and the case where we subtract off the control sample ($2.17$). This improvement is the optimal case for improving on both schemes, as no batch effects will be slightly worse for ours but dramatically better than the "subtract control sample approach" and massive batch effects means that we will slightly underperform subtracting the control sample since measurement noise effect is minimal.

## Batch effects are dominant

Here $\Sigma_\theta=16I$ while $\Sigma_\epsilon=I$. We expect that the traditional subtract the control should do pretty well.

In [5]:
model_b,X_b, controls_b, true_vals_b, batch_effects_b,data_subtracted_b,data_altered_b = run_experiment(2021,
                                                                                        sig_theta=4.0)

100%|█████████████████████████████████████████| 100/100 [01:19<00:00,  1.26it/s]

Ours 2.2178917414250625
Ignored 4.515188343497704
Naive 2.1695650063784666
[0.56617163 0.68738293 0.6946995 ]


As expected, we are basically the same performance with the "subtract the control sample" approach.

## Measurement noise is dominant

In [6]:
model_m,X_m, controls_m, true_vals_m, batch_effects_m,data_subtracted_m,data_altered_m = run_experiment(2021,
                                                                                        sig_theta=.1)

100%|█████████████████████████████████████████| 100/100 [01:17<00:00,  1.28it/s]

Ours 1.1483434451403054
Ignored 1.105450095313051
Naive 2.1695650063784666
[0.15346077 0.18408357 0.22170216]


## Non-iid data.

In our final experiment we evaluate what happens when our data are not iid. In this demonstration we add an effect +3 to the first batch. It doesn't make a difference from the point of view of our algorithm, but presumably if these are all the controls for some disease, this effect is the scientific effect we are trying to detect.


In [7]:
def run_experiment_unbalanced(seed,sig_theta=1.0,sig_eps=1.0):
    X_list, controls, true_vals, batch_effects = simulate_data(seed=seed,sig_theta=sig_theta,sig_eps=sig_eps)
    
    X_list[0] += 3
    true_vals[0] += 3

    model = BatchAdaptationBayesian(n_burn=300,n_samples=300)
    data_altered = model.fit_transform(X_list, controls)

    data_altered_stack = np.vstack(data_altered)
    data_original_stack = np.vstack(X_list)
    true_stack = np.vstack(true_vals)

    data_subtracted = [X_list[i] - controls[i] for i in range(len(X_list))]
    data_subtracted_stack = np.vstack(data_subtracted)

    error_adapted = np.mean((data_altered_stack - true_stack) ** 2)
    error_original = np.mean((data_original_stack - true_stack) ** 2)
    error_subtracted = np.mean((data_subtracted_stack - true_stack) ** 2)
    print('Ours',error_adapted)
    print('Ignored',error_original)
    print('Naive',error_subtracted)
    print(model.w)

    return model,X_list, controls, true_vals, batch_effects,data_subtracted,data_altered


In [9]:
model,X_list, controls, true_vals, batch_effects,data_subtracted,data_altered = run_experiment_unbalanced(2021)

100%|█████████████████████████████████████████| 300/300 [04:06<00:00,  1.21it/s]

Ours 1.6883333202012225
Ignored 1.8987285660395787
Naive 2.1695650063784666
[0.19190466 0.3030366  0.38086262]


Once again we do better. But more importantly, let's make sure the elements in the first batch have higher values than the remaining batches.

In [21]:
for i in range(30):
    print(np.mean((data_altered[i]-true_vals[i])**2),np.mean((X_list[i]-true_vals[i])**2))

1.4210856956845008 1.1856188450942153
1.4907457183862498 1.765418305078454
2.1334387067695877 3.096864905399501
1.5967506145917754 1.6589492561154464
1.1792915076025612 1.5248461281799908
1.3473313531326232 1.3829967689814897
1.3578356137818832 1.3990036675964534
2.7456183304657733 2.1786079685826283
1.3581099261004639 1.5986154703311648
3.736549415774011 4.454555167278858
1.9236541271272218 2.265458896623078
2.793852358074785 3.927230041171649
1.4116084240994589 1.3040300735289696
1.2488766548753532 1.3230051684614392
0.9295318346010354 2.349277579997523
1.9194477202596156 1.789880108727005
1.3845939201456032 1.4911391556596125
1.2686637117054762 1.3134952924493735
0.929216154463235 1.056623922239206
1.3517253338859092 1.2783025086537894
0.938493309192897 0.9110870671616345
1.2632855500983773 1.4558716219132966
1.9529615348411782 2.376652330118831
2.4981870034645626 2.8641941524795747
1.6294809822304175 1.6850747116186982
1.854798048457804 2.6762897721451204
2.5187542278859096 2.08046

In [20]:
np.mean((true_vals[0]-X_list[0])**2)

1.1856188450942153

In [34]:
print('Average value, adapted:',np.mean(data_altered[0]))
print('Average value, truth set:',np.mean(true_vals[0]))
print('Average value, measured:',np.mean(X_list[0]))
print('Average value, combat:',0.000)
print('MSE, adapted:',np.mean((data_altered[0]-true_vals[0])**2))
print('MSE, subtracted:',np.mean((data_subtracted[0]-true_vals[0])**2))
print('MSE, ignored:',np.mean((X_list[0]-true_vals[0])**2))
delta_combat = X_list[0] - np.mean(X_list[0],axis=0)
print('MSE, combat:',np.mean((delta_combat-true_vals[0])**2))


Average value, adapted: 2.366674908926593
Average value, truth set: 2.942432191463585
Average value, measured: 2.3756306286889224
Average value, combat: 0.0
MSE, adapted: 1.4210856956845008
MSE, subtracted: 2.269622494968841
MSE, ignored: 1.1856188450942153
MSE, combat: 9.48405688545448


And to compare to another batch

In [35]:
print('Average value, adapted:',np.mean(data_altered[1]))
print('Average value, truth set:',np.mean(true_vals[1]))
print('Average value, measured:',np.mean(X_list[1]))
print('Average value, combat:',0.000)
print('MSE, adapted:',np.mean((data_altered[1]-true_vals[1])**2))
print('MSE, subtracted:',np.mean((data_subtracted[1]-true_vals[1])**2))
print('MSE, ignored:',np.mean((X_list[1]-true_vals[1])**2))
delta_combat = X_list[1] - np.mean(X_list[1],axis=0)
print('MSE, combat:',np.mean((delta_combat-true_vals[1])**2))

Average value, adapted: 0.3153001867455875
Average value, truth set: 0.03641952421240738
Average value, measured: 0.17833629580401883
Average value, combat: 0.0
MSE, adapted: 1.4907457183862498
MSE, subtracted: 1.557142804918831
MSE, ignored: 1.765418305078454
MSE, combat: 1.0278263839811945


So combat is enormously more effective when the IID assumption is true but introduces massive bias when MSE assumption is violated.

## Conclusions

When the measurement noise is comparable we outperform either naive subtraction or ignoring batch effects. When the noise is either far greater or far less we only have slight performance degredation. But the advantage is that we can estimate the size of these effects. Our method is also robust to imbalances in case/control.